In [3]:
# TODOs: Operate DB Class
# TODOs: Use SQLite
# TODOs: Implement BM25

import sys
from pathlib import Path
sys.path[0] = str(Path(sys.path[0]).parent)

from utils.elastic_db import ElasticDB

# INIT DB OBJECT
PORT = "http://localhost:9200"
INDEX_NAME = "news_cc"

news_db = ElasticDB(elastic_port=PORT, elastic_index=INDEX_NAME)
wiki_db = ElasticDB(elastic_port=PORT, elastic_index="knowledge")

INFO:utils.elastic_db:Connecting to http://localhost:9200 
INFO:utils.elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 
INFO:utils.elastic_db:Connecting to http://localhost:9200 
INFO:utils.elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 


In [34]:
### LOAD DATASETS ###
import json

data = [json.loads(ln) for ln in open("../../data/train_cmv_cleaned.jsonl")]
topics = [json.loads(ln) for ln in open("../../data/claim_topics.jsonl")]

ex_retrieval = [json.loads(ln) for ln in open("../../data/wiki_doc_retrieved_from_op_train.jsonlist")]
ex_ranked = [json.loads(ln) for ln in open("../../data/selected_evidence.jsonl")]

conan = [json.loads(ln) for ln in open("../../data/CONAN.json")]

In [35]:
len(data)

3456

In [36]:
### SAMPLE OUTPUT ###
import random

# TODOs: Replicate Output, Passage Evidence Retrieval
# TODOs: Replicate Output, Passage Ranking

_ = random.randint(0, 1000)
ranked = ex_ranked[_]
retireval = ex_retrieval[_]

retireval

{'tid': 't3_1hrvi8',
 'retrieval_results': [{'retrieved_document_titles': ['Political abuse of psychiatry in the Soviet Union',
    'Optical illusion',
    'Positive illusions',
    'Political positions of Noam Chomsky',
    'Illusion'],
   'query': 'illusions dissent a totalitarian state the usa these illusions characterized overt',
   'sentence': 'The USA is or is becoming a totalitarian state characterized by illusions of choice , freedom , and democracy , as well as overt and physical suppresion of dissent whenever these illusions fail . '},
  {'retrieved_document_titles': ['Police brutality',
    'Police brutality in the United States',
    'Riot shield',
    'Black Panther Party',
    '1967 Detroit riot'],
   'query': 'riot shields police brutality cointelpro stuff',
   'sentence': 'Spying on everybody , domestic drones , police brutality , COINTELPRO stuff , the walls of riot shields at NATO/WTO protests , the list goes on . '},
  {'retrieved_document_titles': ['Holocaust denial

In [37]:
# TODOs: News Data
# TODOs: Ranking
# TODOs: Ranking, Cosine
# TODOs: Research Evidence Retireval: Context Aware, Neural Retrieval
# TODOs: Stance
# TODOs: Target ADUs: Premises, Claims, discard non-ADUs, thus reducing noise over retreival 

# TODOs: Paralellise

In [46]:
from utils.keyphrase_extraction import extract_keyphrase
# from keyphrase_vectorizers import KeyphraseCountVectorizer

import random
import re

_ = random.randint(0, 1000)

arg = data[_]["arguments"]
print(arg)

def to_sentences(text):
    return re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

sentences = to_sentences(arg)
#sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', arg)

queries = []
results = []

for sent in sentences:
    if len(sent) < 5:
        continue

    kp = extract_keyphrase(sent, n_kp=5)
    query = ", ".join(i for i in kp)
    
    # TODOs: Re-init DB with smaller passage size
    titles = [i["_source"]["document"]["title"] for i in news_db.search(query_=query, k=10)]
    evidence = [i["_source"]["document"]["text"] for i in news_db.search(query_=query, k=10)]

    results.append({
        "argument_sentence": sent, 
        "retrieved_documents_titles": titles,
        "query": query,
        "retrieved_evidence": evidence
    })

I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural. He took the position that they are unnatural. He cited this definition by MerriamWebster existing in nature and not made or caused by people coming from nature as his basis for the distinction for natural vs. unnatural.However I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankinds capacity to create problemsolve analyze rationalize and build also come from natural processes. How are the things we create unnatural? It is only through natural occurrences that we have this ability why is it that we would give the credit of these things solely to man as opposed to nature? We are not separate from nature thus how can any of our actions or creations be unnatural? If we were somehow separate from nature I would understand the distinction between natural and manmade. However I think unnatural and manmade

INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.155s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.008s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.068s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.003s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.066s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.006s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.049s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.004s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.065s]
INFO:elastic_transport.transport:POST http://localhost:

In [47]:
### MERGE ###

merged = []
for counter_ev in results:
    sentence = counter_ev["argument_sentence"]
    evidence = counter_ev["retrieved_evidence"]
    sent_kp = counter_ev["query"]

    merged_ev = ", ".join(retrieved for retrieved in evidence)

    merged.append((sentence, sent_kp, merged_ev))

#merged

In [70]:
### PASSAGE RANKING : KEYWORD OVERLAP ###

test = merged[2]
test
sent, kp, merge = test

sent, kp

('He cited this definition by MerriamWebster existing in nature and not made or caused by people coming from nature as his basis for the distinction for natural vs.',
 'nature, merriamwebster, definition, distinction, people')

In [74]:
merged_sentences = to_sentences(merge)
len(merged_sentences)

sent_kps = "".join(i for i in kp)

def count(string, substring):
    n = len(substring)
    cnt = 0
    for i in range(len(string) - n):
        if string[i:i+n] == substring:
            cnt += 1
    return cnt

def overlap_score(count_kp):
    score = 0

    for i in count_kp:
        score += count(sent_kps, count_kp)

    return score

scores = []
for sent in merged_sentences:
    if sent:
        # TODOs: Try-Catch, Handel Errors
        kp = [i for i in extract_keyphrase(sent, n_kp=3)]
        scores.append((kp, overlap_score(kp)))

print(sent_kps)
scores

ValueError: Empty keyphrases. Perhaps the documents do not contain keyphrases that match the 'pos_pattern' parameter, only contain stop words, or you set the 'min_df'/'max_df' parameters too strict.

In [59]:
print(sent_kp)

view, arguments, logic, line


In [ ]:
### ATTACKING PEMISES ###

# from BERT_adu_classifier import predict

# premises = []
# for sent in sentences:
#     prediction = predict(sent)
    
#     if prediction == "premise":
#         premises.append(sent)

In [26]:
# # Argument Level

# results = []

# _ = 90
# hate_ = sample_hate[0]["conan"][_]["hateSpeech"]

# # queries = []
# # sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', hate_)

# results = []

# # TODOs: Refactor key_BERT as Class for multiple oututs
# #kp_ = (exctract_keyphrase(hate_, n_gram=1, n_kp=1), exctract_keyphrase(hate_, n_gram=2, n_kp=1), exctract_keyphrase(hate_, n_gram=3, n_kp=1))

# kp_ = (exctract_keyphrase(hate_, n_gram=5, n_kp=1))

# query = " ,".join(i for i in kp_)
# query

# res = [i["_source"]["document"]["title"] for i in news_db.search(query_=query, k=5)]

# results.append({
#     "argument_sentence": hate_, 
#     "query": query, 
#     "retireved_documents": res,

# })

# results


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.367s]


[{'argument_sentence': 'Fuck Islam and Fuck ISIS. We should repatriate all Muslims. They are all a bunch of violent criminals who do not share our values and never will. Without them Britain will be a better, safer place.',
  'query': 'fuck isis repatriate muslims',
  'retireved_documents': ['‘Assalamu Alaykom:’ peace be upon you',
   'How We Got to Here: A Charlottesville Reading List',
   'De-radicalization can work for former ISIS fighters',
   'How to Get Assyrians to Stay',
   'How to Get Christians to Stay']}]

In [27]:
# results = []
# res = [i["_source"]["document"]["title"] for i in wiki_db.search(query_=query, k=5)]

# results.append({
#     "argument_sentence": hate_, 
#     "query": query, 
#     "retireved_documents": res,

# })

# results

INFO:elastic_transport.transport:POST http://localhost:9200/knowledge/_search [status:200 duration:0.249s]


[{'argument_sentence': 'Fuck Islam and Fuck ISIS. We should repatriate all Muslims. They are all a bunch of violent criminals who do not share our values and never will. Without them Britain will be a better, safer place.',
  'query': 'fuck isis repatriate muslims',
  'retireved_documents': ['Refuting ISIS',
   'Refuting ISIS',
   'Take Off (2017 film)',
   'Terrorism in India',
   'Tareq Kamleh']}]

In [ ]:
# #queries
# query = "the executive the presidential system legislative branches partisan unproductive"
# res = search_text(wiki_ev, query_=query, k=3)
# res

[{'_index': 'knowledge',
  '_id': 'kbZa5YIBRqC1428DaQUk',
  '_score': 33.672318,
  '_ignored': ['document.text.keyword'],
  '_source': {'document': {'id': '3408791',
    'source': 'wikipedia',
    'title': 'Ruling party',
    'text': "In parliamentary systems, the majority in the legislature also controls the executive branch of government, thus leaving no possibility of opposing parties concurrently occupying the executive and legislative branches of government. In other systems, such as in an American style presidential system, the party of the president does not necessarily also have a legislative majority. A 'ruling party' is also used to describe the party of one-party states, such as the Chinese Communist Party in the People's Republic of China. In his political manifesto 'The Green Book', the late Libyan leader Muammar al-Gaddafi attacked the ability of the ruling party, using it as a basis for his opposition to partisan politics. None"}}},
 {'_index': 'knowledge',
  '_id': 'JQ3

In [ ]:
# import re

# print(", ".join(re.sub(r'\[\[(?:[^|\]]*\|)?([^\]]*)]]', "", i["_source"]["document"]["title"]).strip("[]") for i in res))

Ruling party, Fusion of powers, Student governments in the United States


In [ ]:
# ### TEST SEARCH ###

# def search_text(es, query_, k=5):
#     results = es.search(
#         index = es.elastic_index,
#         query = {
#             "match": {
#                 "document.text": query_,
#                 },
#         },
#         size=k)

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     return hits

# def search_topic(es, topic, k=5):
#     results = es.search(
#         index = es.elastic_index,
#         body= {
#             "size": k,
#             "query": {
#                 "match": {
#                     "document.title": topic,
#         }}})

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     title = hits[0]["_source"]["document"]["title"]
#     text = hits[0]["_source"]["document"]["text"]

#     return {
#         "title": title,
#         "text": text
#     }

# query = "government emails"
# text = search_text(wiki_ev, query_=query, k=10)

# print(text)


In [ ]:
### SEARCH FUNCTION ###

# TODOs: Implement as Class-DB object

# def search_text(db, query_, k=5):
#     results = db.search(
#         index = es.elastic_index,
#         query = {
#             "match": {
#                 "document.text": query_,
#                 },
#         },
#         size=k)

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     return hits